In [1]:
# Install the following packages incase they're not loaded already
!pip install pdfplumber
!pip install chromadb
!pip install tiktoken
!pip install openai

   ---------------------------------------- 0.0/5.6 MB ? eta -:--:--
   --- ------------------------------------ 0.5/5.6 MB 5.6 MB/s eta 0:00:01
   ----------- ---------------------------- 1.6/5.6 MB 4.9 MB/s eta 0:00:01
   ------------------ --------------------- 2.6/5.6 MB 4.6 MB/s eta 0:00:01
   --------------------------- ------------ 3.9/5.6 MB 4.9 MB/s eta 0:00:01
   ----------------------------------- ---- 5.0/5.6 MB 5.0 MB/s eta 0:00:01
   ---------------------------------------- 5.6/5.6 MB 5.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.9 MB ? eta -:--:--
   ------------------ --------------------- 1.3/2.9 MB 6.7 MB/s eta 0:00:01
   -------------------------------- ------- 2.4/2.9 MB 5.8 MB/s eta 0:00:01
   ---------------------------------------- 2.9/2.9 MB 5.4 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to bui

  error: subprocess-exited-with-error
  
  exit code: 1
  
  [5 lines of output]
  running bdist_wheel
  running build
  running build_ext
  building 'hnswlib' extension
  error: Microsoft Visual C++ 14.0 or greater is required. Get it with "Microsoft C++ Build Tools": https://visualstudio.microsoft.com/visual-cpp-build-tools/
  [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for chroma-hnswlib
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (chroma-hnswlib)


   ---------------------------------------- 0.0/878.7 kB ? eta -:--:--
   ----------------------------------- ---- 786.4/878.7 kB 5.6 MB/s eta 0:00:01
   ---------------------------------------- 878.7/878.7 kB 4.9 MB/s eta 0:00:00


In [6]:
!pip install chromadb

  Using cached build-1.3.0-py3-none-any.whl.metadata (5.6 kB)
  Using cached uvicorn-0.38.0-py3-none-any.whl.metadata (6.8 kB)
  Using cached onnxruntime-1.23.2-cp312-cp312-win_amd64.whl.metadata (5.3 kB)
  Using cached opentelemetry_api-1.38.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached opentelemetry_exporter_otlp_proto_grpc-1.38.0-py3-none-any.whl.metadata (2.4 kB)
  Using cached opentelemetry_sdk-1.38.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached tokenizers-0.22.1-cp39-abi3-win_amd64.whl.metadata (6.9 kB)
  Using cached pypika-0.48.9-py2.py3-none-any.whl
  Using cached importlib_resources-6.5.2-py3-none-any.whl.metadata (3.9 kB)
  Using cached bcrypt-5.0.0-cp39-abi3-win_amd64.whl.metadata (10 kB)
  Using cached kubernetes-34.1.0-py2.py3-none-any.whl.metadata (1.7 kB)
  Using cached mmh3-5.2.0-cp312-cp312-win_amd64.whl.metadata (14 kB)
  Using cached orjson-3.11.4-cp312-cp312-win_amd64.whl.metadata (42 kB)
  Using cached pyproject_hooks-1.2.0-py3-none-any.whl.metadata (1.

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-intel 2.18.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<6.0.0dev,>=3.20.3, but you have protobuf 6.33.0 which is incompatible.


In [12]:
# Import all the required Libraries
import pdfplumber
from pathlib import Path
import pandas as pd
from operator import itemgetter
import json
import tiktoken
import chromadb
import openai

In [13]:
filepath = r"C:\Users\shrey\OneDrive\Desktop\OpenAPIKey.txt"

In [14]:
# Set the API key
with open(filepath , "r") as f:
  openai.api_key = ' '.join(f.readlines())

### Data Processing

Quite crucial, hence suggested to use good parsers that can load the data from your documents effectively.


In [15]:
pdf_path = r"C:\Users\shrey\OneDrive\Shreya_C71\HelpMate_AI\Principal-Sample-Life-Insurance-Policy.pdf"

In [16]:
# Function to check whether a word is present in a table or not for segregation of regular text and tables

def check_bboxes(word, table_bbox):
    # Check whether word is inside a table bbox.
    l = word['x0'], word['top'], word['x1'], word['bottom']
    r = table_bbox
    return l[0] > r[0] and l[1] > r[1] and l[2] < r[2] and l[3] < r[3]

In [17]:
# Function to extract text from a PDF file.
# 1. Declare a variable p to store the iteration of the loop that will help us store page numbers alongside the text
# 2. Declare an empty list 'full_text' to store all the text files
# 3. Use pdfplumber to open the pdf pages one by one
# 4. Find the tables and their locations in the page
# 5. Extract the text from the tables in the variable 'tables'
# 6. Extract the regular words by calling the function check_bboxes() and checking whether words are present in the table or not
# 7. Use the cluster_objects utility to cluster non-table and table words together so that they retain the same chronology as in the original PDF
# 8. Declare an empty list 'lines' to store the page text
# 9. If a text element in present in the cluster, append it to 'lines', else if a table element is present, append the table
# 10. Append the page number and all lines to full_text, and increment 'p'
# 11. When the function has iterated over all pages, return the 'full_text' list

def extract_text_from_pdf(pdf_path):
    p = 0
    full_text = []


    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            page_no = f"Page {p+1}"
            text = page.extract_text()

            tables = page.find_tables()
            table_bboxes = [i.bbox for i in tables]
            tables = [{'table': i.extract(), 'top': i.bbox[1]} for i in tables]
            non_table_words = [word for word in page.extract_words() if not any([check_bboxes(word, table_bbox) for table_bbox in table_bboxes])]
            lines = []

            for cluster in pdfplumber.utils.cluster_objects(non_table_words + tables, itemgetter('top'), tolerance=5):

                if 'text' in cluster[0]:
                    try:
                        lines.append(' '.join([i['text'] for i in cluster]))
                    except KeyError:
                        pass

                elif 'table' in cluster[0]:
                    lines.append(json.dumps(cluster[0]['table']))


            full_text.append([page_no, " ".join(lines)])
            p +=1

    return full_text

In [18]:
# Define the directory containing the PDF files
pdf_directory = Path(pdf_path)

# Process the PDF file
print(f"...Processing {pdf_path}")

# Call your extraction function (must return a list of [page_number, text])
extracted_text = extract_text_from_pdf(pdf_path)

# Convert the extracted list to a DataFrame
insurance_pdfs_data = pd.DataFrame(extracted_text, columns=['Page No.', 'Page_Text'])

# Show sample output
print(f"Finished processing {pdf_path}")
print(insurance_pdfs_data.head(10))

...Processing C:\Users\shrey\OneDrive\Shreya_C71\HelpMate_AI\Principal-Sample-Life-Insurance-Policy.pdf
Finished processing C:\Users\shrey\OneDrive\Shreya_C71\HelpMate_AI\Principal-Sample-Life-Insurance-Policy.pdf
  Page No.                                          Page_Text
0   Page 1  DOROTHEA GLAUSE S655 RHODE ISLAND JOHN DOE 01/...
1   Page 2                 This page left blank intentionally
2   Page 3  POLICY RIDER GROUP INSURANCE POLICY NO: S655 C...
3   Page 4                 This page left blank intentionally
4   Page 5  PRINCIPAL LIFE INSURANCE COMPANY (called The P...
5   Page 6  TABLE OF CONTENTS PART I - DEFINITIONS PART II...
6   Page 7  Section A – Eligibility Member Life Insurance ...
7   Page 8  Section A - Member Life Insurance Schedule of ...
8   Page 9  P ART I - DEFINITIONS When used in this Group ...
9  Page 10  T he legally recognized union of two eligible ...


In [19]:
insurance_pdfs_data

,Page No.,Page_Text
0,Page 1,DOROTHEA GLAUSE S655 RHODE ISLAND JOHN DOE 01/...
1,Page 2,This page left blank intentionally
2,Page 3,POLICY RIDER GROUP INSURANCE POLICY NO: S655 C...
3,Page 4,This page left blank intentionally
4,Page 5,PRINCIPAL LIFE INSURANCE COMPANY (called The P...
...,...,...
59,Page 60,I f a Dependent who was insured dies during th...
60,Page 61,Section D - Claim Procedures Article 1 - Notic...
61,Page 62,A claimant may request an appeal of a claim de...
62,Page 63,This page left blank intentionally


In [20]:
insurance_pdfs_data.Page_Text[2]

'POLICY RIDER GROUP INSURANCE POLICY NO: S655 COVERAGE: Life EMPLOYER: RHODE ISLAND JOHN DOE Effective on the later of the Date of Issue of this Group Policy or March 1, 2005, the following will apply to your Policy: From time to time The Principal may offer or provide certain employer groups who apply for coverage with The Principal a Financial Services Hotline and Grief Support Services or any other value added service for the employees of that employer group. In addition, The Principal may arrange for third party service providers (i.e., optometrists, health clubs), to provide discounted goods and services to those employer groups who apply for coverage with The Principal or who become insureds/enrollees of The Principal. While The Principal has arranged these goods, services and/or third party provider discounts, the third party service providers are liable to the applicants/insureds/enrollees for the provision of such goods and/or services. The Principal is not responsible for the

In [21]:
len(insurance_pdfs_data)

64

In [22]:
insurance_pdfs_data['Text_Length'] = insurance_pdfs_data['Page_Text'].apply(lambda x: len(x.split(' ')))

In [23]:
insurance_pdfs_data['Text_Length']

0      30
1       5
2     230
3       5
4     110
     ... 
59    285
60    418
61    322
62      5
63      8
Name: Text_Length, Length: 64, dtype: int64

In [24]:
# Retain only the rows with a text length of at least 5 to exclue the intentionally left blank pages

insurance_pdfs_data = insurance_pdfs_data.loc[insurance_pdfs_data['Text_Length'] >= 6]
insurance_pdfs_data

,Page No.,Page_Text,Text_Length
0,Page 1,DOROTHEA GLAUSE S655 RHODE ISLAND JOHN DOE 01/...,30
2,Page 3,POLICY RIDER GROUP INSURANCE POLICY NO: S655 C...,230
4,Page 5,PRINCIPAL LIFE INSURANCE COMPANY (called The P...,110
5,Page 6,TABLE OF CONTENTS PART I - DEFINITIONS PART II...,153
6,Page 7,Section A – Eligibility Member Life Insurance ...,176
...,...,...,...
58,Page 59,Section C - Dependent Life Insurance Article 1...,240
59,Page 60,I f a Dependent who was insured dies during th...,285
60,Page 61,Section D - Claim Procedures Article 1 - Notic...,418
61,Page 62,A claimant may request an appeal of a claim de...,322


In [25]:
insurance_pdfs_data['Metadata'] = insurance_pdfs_data.apply(
    lambda x: {
        'Page_No': x['Page No.'],
        'Text_Length': x['Text_Length'],
        'Preview_Text': x['Page_Text'][:100]  # optional: include a short snippet
    },
    axis=1
)

C:\Users\shrey\AppData\Local\Temp\ipykernel_27956\294067445.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  insurance_pdfs_data['Metadata'] = insurance_pdfs_data.apply(


In [26]:
insurance_pdfs_data.head()

,Page No.,Page_Text,Text_Length,Metadata
0,Page 1,DOROTHEA GLAUSE S655 RHODE ISLAND JOHN DOE 01/...,30,"{'Page_No': 'Page 1', 'Text_Length': 30, 'Prev..."
2,Page 3,POLICY RIDER GROUP INSURANCE POLICY NO: S655 C...,230,"{'Page_No': 'Page 3', 'Text_Length': 230, 'Pre..."
4,Page 5,PRINCIPAL LIFE INSURANCE COMPANY (called The P...,110,"{'Page_No': 'Page 5', 'Text_Length': 110, 'Pre..."
5,Page 6,TABLE OF CONTENTS PART I - DEFINITIONS PART II...,153,"{'Page_No': 'Page 6', 'Text_Length': 153, 'Pre..."
6,Page 7,Section A – Eligibility Member Life Insurance ...,176,"{'Page_No': 'Page 7', 'Text_Length': 176, 'Pre..."


In [27]:
# Import the OpenAI Embedding Function into chroma
from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction

In [28]:
# Define the path where chroma collections will be stored

chroma_data_path = r"C:\Users\shrey\OneDrive\Shreya_C71\HelpMate_AI"

In [29]:
# Call PersistentClient()

client = chromadb.PersistentClient(path=chroma_data_path)

In [30]:
# Set up the embedding function using the OpenAI embedding model

model = "text-embedding-3-small" 
embedding_function = OpenAIEmbeddingFunction(api_key=openai.api_key, model_name=model)

In [31]:
## Create an empty collection
insurance_collection = client.get_or_create_collection(name='RAG_on_Policy', embedding_function=embedding_function)

In [32]:
documents_list = insurance_pdfs_data["Page_Text"].tolist()
metadata_list = insurance_pdfs_data['Metadata'].tolist()

In [33]:
# Add the documents and metadata to the collection alongwith generic integer IDs. You can also feed the metadata information as IDs by combining the policy name and page no.
insurance_collection.add(
    documents= documents_list,
    ids = [str(i) for i in range(0, len(documents_list))],
    metadatas = metadata_list
)

In [34]:

insurance_collection.get(
    ids = ['0','1','2'],
    include = ['embeddings', 'documents', 'metadatas']
)

{'ids': ['0', '1', '2'],
 'embeddings': array([[ 0.02404991, -0.00253496,  0.05455546, ...,  0.00448125,
         -0.01203157, -0.01768687],
        [ 0.02752786,  0.02625884,  0.04566013, ...,  0.00913935,
         -0.00302306, -0.0028202 ],
        [ 0.02293694,  0.02192519,  0.05537242, ...,  0.02785285,
          0.01055195,  0.00303971]]),
 'documents': ['DOROTHEA GLAUSE S655 RHODE ISLAND JOHN DOE 01/01/2014 711 HIGH STREET GEORGE RI 02903 GROUP POLICY FOR: RHODE ISLAND JOHN DOE ALL MEMBERS Group Member Life Insurance Print Date: 07/16/2014',
  'POLICY RIDER GROUP INSURANCE POLICY NO: S655 COVERAGE: Life EMPLOYER: RHODE ISLAND JOHN DOE Effective on the later of the Date of Issue of this Group Policy or March 1, 2005, the following will apply to your Policy: From time to time The Principal may offer or provide certain employer groups who apply for coverage with The Principal a Financial Services Hotline and Grief Support Services or any other value added service for the employees o

In [35]:
cache_collection = client.get_or_create_collection(name='Insurance_Cache', embedding_function=embedding_function)

In [36]:
cache_collection.peek()

{'ids': ['What is the claim process in terms of accidental death',
  'In what conditions the policy can be terminated',
  'What are conditions to form Civil Union'],
 'embeddings': array([[ 0.04070305,  0.04031854,  0.01422684, ...,  0.02363359,
          0.02751988,  0.02059871],
        [ 0.00777198,  0.07250334,  0.03601456, ..., -0.00681036,
         -0.00284369, -0.00218999],
        [ 0.0161234 ,  0.02060069,  0.04919837, ..., -0.00165553,
         -0.03110807,  0.04611862]]),
 'documents': ['What is the claim process in terms of accidental death',
  'In what conditions the policy can be terminated',
  'What are conditions to form Civil Union'],
 'uris': None,
 'included': ['metadatas', 'documents', 'embeddings'],
 'data': None,
 'metadatas': [{'distances_4': '1.0037035942077637',
   'included_2': 't',
   'metadatas_1': "{'Preview_Text': 'f . claim requirements listed in PART IV, Section D, must be satisfied; and g. all medical evidence ', 'Text_Length': 368, 'Page_No': 'Page 54'

## Semantic Search with Cache

We have created 2 collections
- insurance_collection (embeddings are already present)
- cache_collection (an empty collection at the beginning)

Our strategy is to first check the cache_collection and see if can answer the query or not.

In case it doesn't the query then passes to the financedata_collection, and the query results get appended to cache_collection (embeddings, metadata, etc)

Semantic Search Pipeline

In [72]:
# Read the user query
query = input()

 What is the claim process in terms of accidental death


In [73]:
cache_results = cache_collection.query(
    query_texts=query,
    n_results=1
)

In [74]:
cache_results

{'ids': [['What is the claim process in terms of accidental death']],
 'embeddings': None,
 'documents': [['What is the claim process in terms of accidental death']],
 'uris': None,
 'included': ['metadatas', 'documents', 'distances'],
 'data': None,
 'metadatas': [[{'documents_9': "M ember's death, the Death Benefits Payable may be withheld until additional information has been received or the trial has been held. If a Member who was insured dies within the 31-day individual purchase period described in PART III, Section F, The Principal will pay his or her beneficiary the individual policy amount, if any, the Member had the right to purchase. No payment will be made before The Principal receives Written proof of the Member's death. Article 3 - Beneficiary A beneficiary should be named at the time a Member applies or enrolls under this Group Policy. A Member may name or later change a named beneficiary by sending a Written request to The Principal. A change will not be effective until

In [75]:
results = insurance_collection.query(
query_texts=query,
n_results=10
)

In [41]:
results

{'ids': [['7', '29', '6', '8', '24', '23', '30', '34', '14', '38']],
 'embeddings': None,
 'documents': [['T he legally recognized union of two eligible individuals of the same sex established according to law. Civil Union Partner For two persons to establish a Civil Union in Rhode Island, it shall be necessary that they satisfy all of the following criteria: a. not be a party to another Civil Union or marriage in Rhode Island; b. be of the same sex and therefore be excluded from the marriage laws of Rhode Island or any other state; c. be at least 18 years of age; d. not be related to the other proposed party to the Civil Union. NOTE: For the purposes of this Group Policy, the term "spouse" will include Civil Union Partner, except as otherwise provided in this Group Policy. Date of Issue The date this Group Policy is placed in force: November 1, 2007. Dependent a. A Member\'s spouse, if that spouse: (1) is legally married to the Member; and (2) is not in the Armed Forces of any country

In [76]:
# Set a threshold for cache search
# Set a threshold for cache search
threshold = 0.2

ids = []
documents = []
distances = []
metadatas = []
results_df = pd.DataFrame()

# If the distance is greater than the threshold, then return the results from the main collection.
if cache_results['distances'][0] == [] or cache_results['distances'][0][0] > threshold:
    # Query the main collection
    results = insurance_collection.query(
        query_texts=query,
        n_results=10
    )

    # Prepare keys and values for metadata storage
    Keys = []
    Values = []

    # Each 'results[key]' is a list of lists, so iterate correctly
    for key, val in results.items():
        if val is None:
            continue

        # Handle nested list structure safely
        inner_list = val[0] if isinstance(val, list) and len(val) > 0 else []
        for i, item in enumerate(inner_list):
            Keys.append(f"{key}_{i}")
            Values.append(str(item))

    # Add the new query result to cache collection
    cache_collection.add(
        documents=[query],
        ids=[query],  # or unique id if preferred
        metadatas=dict(zip(Keys, Values))
    )

    print("Not found in cache. Found in main collection.")

    # Build a dataframe of results
    result_dict = {
        'Metadatas': results['metadatas'][0],
        'Documents': results['documents'][0],
        'Distances': results['distances'][0],
        'IDs': results['ids'][0]
    }

    results_df = pd.DataFrame.from_dict(result_dict)

# If the distance is, however, less than the threshold, return from cache
elif cache_results['distances'][0][0] <= threshold:
    cache_result_dict = cache_results['metadatas'][0][0]

    # Extract ids, docs, distances, metadatas
    for key, value in cache_result_dict.items():
        if 'ids' in key:
            ids.append(value)
        elif 'documents' in key:
            documents.append(value)
        elif 'distances' in key:
            distances.append(value)
        elif 'metadatas' in key:
            metadatas.append(value)

    print("Found in cache!")

    results_df = pd.DataFrame({
        'IDs': ids,
        'Documents': documents,
        'Distances': distances,
        'Metadatas': metadatas
    })

results_df


Found in cache!


,IDs,Documents,Distances,Metadatas
0,54,"M ember's death, the Death Benefits Payable ma...",1.0333735942840576,"{'Text_Length': 273, 'Page_No': 'Page 51', 'Pr..."
1,5,"f . claim requirements listed in PART IV, Sect...",1.0550814867019653,"{'Page_No': 'Page 53', 'Preview_Text': 'Sectio..."
2,48,"If a Member sustains an injury, and as a resul...",1.0567841529846191,"{'Text_Length': 321, 'Preview_Text': '% of Sch..."
3,59,Section A - Member Life Insurance Schedule of ...,1.017758846282959,"{'Preview_Text': ""M ember's death, the Death B..."
4,53,A claimant may request an appeal of a claim de...,1.0683801174163818,"{'Text_Length': 171, 'Page_No': 'Page 8', 'Pre..."
5,44,Coverage During Disability will cease on the e...,0.8733044862747192,{'Preview_Text': 'Exposure Exposure to the ele...
6,52,Section B - Member Accidental Death and Dismem...,0.8689153790473938,{'Preview_Text': 'f . claim requirements liste...
7,50,% of Scheduled Covered Loss Benefit Loss of Sp...,1.0037035942077637,{'Preview_Text': 'Section D - Claim Procedures...
8,51,Exposure Exposure to the elements will be pres...,0.9040501117706299,"{'Text_Length': 322, 'Preview_Text': 'A claima..."
9,58,Section D - Claim Procedures Article 1 - Notic...,0.9493213891983032,"{'Page_No': 'Page 56', 'Text_Length': 307, 'Pr..."


In [3]:
!pip install sentence_transformers

In [4]:
import sys
print(sys.executable)

C:\Users\shrey\anaconda3\python.exe


In [118]:
!{sys.executable} -m pip install --user tf-keras

  Using cached tf_keras-2.20.1-py3-none-any.whl.metadata (1.8 kB)
  Using cached tensorflow-2.20.0-cp312-cp312-win_amd64.whl.metadata (4.6 kB)
Using cached tf_keras-2.20.1-py3-none-any.whl (1.7 MB)
Using cached tensorflow-2.20.0-cp312-cp312-win_amd64.whl (331.9 MB)


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [1]:
# Import the CrossEncoder library from sentence_transformers
from sentence_transformers import CrossEncoder, util

In [5]:
!{sys.executable} -m pip install ipywidgets --upgrade

   ---------------------------------------- 0.0/2.2 MB ? eta -:--:--
   ----------------------- ---------------- 1.3/2.2 MB 7.5 MB/s eta 0:00:01
   ---------------------------------------- 2.2/2.2 MB 5.9 MB/s eta 0:00:00
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 3.6.6
    Uninstalling widgetsnbextension-3.6.6:
      Successfully uninstalled widgetsnbextension-3.6.6
  Attempting uninstall: jupyterlab_widgets
    Found existing installation: jupyterlab-widgets 1.0.0
    Uninstalling jupyterlab-widgets-1.0.0:
      Successfully uninstalled jupyterlab-widgets-1.0.0
  Attempting uninstall: ipywidgets
    Found existing installation: ipywidgets 7.8.1
    Uninstalling ipywidgets-7.8.1:
      Successfully uninstalled ipywidgets-7.8.1


In [6]:
# Initialise the cross encoder model

cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

In [7]:
# Test the cross encoder model

scores = cross_encoder.predict([['What is the claim process in terms of accidental death?', 'Written notice must be sent to the Principal within 20 days after loss'],
                                ['What is the claim process in terms of accidental death?', 'Claim cannot be processed based on incomplete information']])

In [8]:
scores

array([-11.313068,  -5.682571], dtype=float32)

In [77]:
# Input (query, response) pairs for each of the top 20 responses received from the semantic search to the cross encoder
# Generate the cross_encoder scores for these pairs

cross_inputs = [[query, response] for response in results_df['Documents']]
cross_rerank_scores = cross_encoder.predict(cross_inputs)

In [78]:
 #Store the rerank_scores in results_df

results_df['Reranked_scores'] = cross_rerank_scores

In [79]:
results_df

,IDs,Documents,Distances,Metadatas,Reranked_scores
0,54,"M ember's death, the Death Benefits Payable ma...",1.0333735942840576,"{'Text_Length': 273, 'Page_No': 'Page 51', 'Pr...",-7.234458
1,5,"f . claim requirements listed in PART IV, Sect...",1.0550814867019653,"{'Page_No': 'Page 53', 'Preview_Text': 'Sectio...",-2.629317
2,48,"If a Member sustains an injury, and as a resul...",1.0567841529846191,"{'Text_Length': 321, 'Preview_Text': '% of Sch...",-4.344800
3,59,Section A - Member Life Insurance Schedule of ...,1.017758846282959,"{'Preview_Text': ""M ember's death, the Death B...",-4.123010
4,53,A claimant may request an appeal of a claim de...,1.0683801174163818,"{'Text_Length': 171, 'Page_No': 'Page 8', 'Pre...",-7.225667
5,44,Coverage During Disability will cease on the e...,0.8733044862747192,{'Preview_Text': 'Exposure Exposure to the ele...,-11.009520
6,52,Section B - Member Accidental Death and Dismem...,0.8689153790473938,{'Preview_Text': 'f . claim requirements liste...,-3.893980
7,50,% of Scheduled Covered Loss Benefit Loss of Sp...,1.0037035942077637,{'Preview_Text': 'Section D - Claim Procedures...,-4.620596
8,51,Exposure Exposure to the elements will be pres...,0.9040501117706299,"{'Text_Length': 322, 'Preview_Text': 'A claima...",-3.561475
9,58,Section D - Claim Procedures Article 1 - Notic...,0.9493213891983032,"{'Page_No': 'Page 56', 'Text_Length': 307, 'Pr...",-6.551778


In [80]:
# Return the top 3 results from semantic search
top_3_semantic = results_df.sort_values(by='Distances')
top_3_semantic[:3]

,IDs,Documents,Distances,Metadatas,Reranked_scores
6,52,Section B - Member Accidental Death and Dismem...,0.8689153790473938,{'Preview_Text': 'f . claim requirements liste...,-3.893980
5,44,Coverage During Disability will cease on the e...,0.8733044862747192,{'Preview_Text': 'Exposure Exposure to the ele...,-11.009520
8,51,Exposure Exposure to the elements will be pres...,0.9040501117706299,"{'Text_Length': 322, 'Preview_Text': 'A claima...",-3.561475


In [81]:
# Return the top 3 results after reranking

top_3_rerank = results_df.sort_values(by='Reranked_scores', ascending=False)
top_3_rerank[:3]
top_3_RAG = top_3_rerank[["Documents", "Metadatas"]][:3]

In [82]:
top_3_RAG

,Documents,Metadatas
1,"f . claim requirements listed in PART IV, Sect...","{'Page_No': 'Page 53', 'Preview_Text': 'Sectio..."
8,Exposure Exposure to the elements will be pres...,"{'Text_Length': 322, 'Preview_Text': 'A claima..."
6,Section B - Member Accidental Death and Dismem...,{'Preview_Text': 'f . claim requirements liste...


In [83]:
top_3_RAG['Documents'].iloc[0]

"f . claim requirements listed in PART IV, Section D, must be satisfied; and g. all medical evidence must be satisfactory to The Principal. Article 3 - Benefits Payable If all of the benefit qualifications are met, The Principal will pay: a. 100% of the Scheduled Benefit (or approved amount, if applicable) in force for loss of life; or b. 50% of the Scheduled Benefit (or approved amount, if applicable) in force if one hand is severed at or above the wrist; or c. 25% of the Scheduled Benefit (or approved amount, if applicable) in force for loss of thumb and index finger on the same hand; or d. 50% of the Scheduled Benefit (or approved amount, if applicable) in force if one foot is severed at or above the ankle; or e. 50% of the Scheduled Benefit (or approved amount, if applicable) in force if the sight of one eye is permanently lost (For this purpose, vision not correctable to better than 20/200 will be considered loss of sight.); or f. 100% of the Scheduled Benefit (or approved amount,

## Build the RAG system

In [84]:
# Define the function to generate the response. Provide a comprehensive prompt that passes the user query and the top 3 results to the model

def generate_response(query, results_df):
    """
    Generate a response using GPT-3.5's ChatCompletion based on the user query and retrieved information.
    """
    messages = [
                {"role": "system", "content":  "You are a helpful assistant in the insurance domain who can effectively answer user queries about insurance policies and documents."},
                {"role": "user", "content": f"""You are a helpful assistant in the insurance domain who can effectively answer user queries about insurance policies and documents.
                                                You have a question asked by the user in '{query}' and you have some search results from a corpus of insurance documents in the dataframe '{top_3_RAG}'. These search results are essentially one page of an insurance document that may be relevant to the user query.

                                                The column 'documents' inside this dataframe contains the actual text from the policy document and the column 'metadata' contains the policy name and source page. The text inside the document may also contain tables in the format of a list of lists where each of the nested lists indicates a row.

                                                Use the documents in '{top_3_RAG}' to answer the query '{query}'. Frame an informative answer and also, use the dataframe to return the relevant policy names and page numbers as citations.

                                                Follow the guidelines below when performing the task.
                                                1. Try to provide relevant/accurate numbers if available.
                                                2. You don’t have to necessarily use all the information in the dataframe. Only choose information that is relevant.
                                                3. If the document text has tables with relevant information, please reformat the table and return the final information in a tabular in format.
                                                3. Use the Metadatas columns in the dataframe to retrieve and cite the policy name(s) and page numbers(s) as citation.
                                                4. If you can't provide the complete answer, please also provide any information that will help the user to search specific sections in the relevant cited documents.
                                                5. You are a customer facing assistant, so do not provide any information on internal workings, just answer the query directly.

                                                The generated response should answer the query directly addressing the user and avoiding additional information. If you think that the query is not relevant to the document, reply that the query is irrelevant. Provide the final response as a well-formatted and easily readable text along with the citation. Provide your complete response first with all information, and then provide the citations.
                                                """},
              ]

    response = openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages
    )

    return response.choices[0].message.content.split('\n')

In [85]:
response = generate_response(query, top_3_RAG)

In [86]:
print("\n".join(response))

The claim process for accidental death is outlined in Section B of the policy document. It specifies the requirements and steps for initiating a claim in case of accidental death. The document provides detailed information on how to proceed with the claim submission and the essential documents needed for processing the claim. For specific details on the claim process, please refer to the policy document Section B - Member Accidental Death and Dismemberment.

Citation:
- Policy Name: Section B - Member Accidental Death and Dismemberment
- Page Number: Page 53
